# Julia Parallelism Basics

In [1]:
addprocs(2)

2-element Array{Int64,1}:
 2
 3

In [2]:
nworkers()

2

In [3]:
workers()

2-element Array{Int64,1}:
 2
 3

In [4]:
r = remotecall(rand, 2, 2, 2)

Future(2, 1, 4, Nullable{Any}())

In [5]:
 fetch(r)

2×2 Array{Float64,2}:
 0.197288  0.629904
 0.328007  0.315113

In [6]:
s = @spawnat 3 1+fetch(r)

Future(3, 1, 6, Nullable{Any}())

In [7]:
fetch(s)

2×2 Array{Float64,2}:
 1.19729  1.6299 
 1.32801  1.31511

In [8]:
remotecall_fetch(getindex, 2, r, 1, 1)

0.19728790416018938

In [9]:
r = @spawn rand(2,2)

Future(2, 1, 9, Nullable{Any}())

In [10]:
s = @spawn 1+fetch(r)

Future(3, 1, 10, Nullable{Any}())

In [11]:
function rand2(dims...)
    return 2*rand(dims...)
end

rand2 (generic function with 1 method)

In [12]:
rand2(2,2)

2×2 Array{Float64,2}:
 1.81614   1.17952
 0.583024  1.99303

In [13]:
r2 = @spawn rand2(2,2)

Future(2, 1, 11, Nullable{Any}())

In [14]:
fetch(r2)

LoadError: [91mOn worker 2:
[91mUndefVarError: #rand2 not defined[39m
deserialize_datatype at ./serialize.jl:973
handle_deserialize at ./serialize.jl:677
deserialize at ./serialize.jl:637
handle_deserialize at ./serialize.jl:684
deserialize_global_from_main at ./distributed/clusterserialize.jl:154
foreach at ./abstractarray.jl:1733
deserialize at ./distributed/clusterserialize.jl:56
handle_deserialize at ./serialize.jl:726
deserialize at ./serialize.jl:637
handle_deserialize at ./serialize.jl:681
deserialize at ./serialize.jl:637
handle_deserialize at ./serialize.jl:684
deserialize_msg at ./distributed/messages.jl:98
message_handler_loop at ./distributed/process_messages.jl:161
process_tcp_streams at ./distributed/process_messages.jl:118
#99 at ./event.jl:73[39m

In [15]:
@everywhere function rand2(dims...)
    return 2*rand(dims...)
end

In [16]:
r2 = @spawn rand2(2,2)
fetch(r2)

2×2 Array{Float64,2}:
 0.48609   1.06869 
 0.275809  0.241942

## Data Movement

In [34]:
@time begin 
    A = rand(1000,1000)
    Bref = @spawn A^2
    fetch(Bref)
end

  0.209178 seconds (323 allocations: 15.270 MiB, 49.92% gc time)


1000×1000 Array{Float64,2}:
 259.909  250.942  256.955  253.812  …  252.901  250.257  265.245  258.605
 257.849  248.031  254.137  259.737     255.823  247.793  261.132  255.321
 266.534  257.839  257.644  258.331     257.45   252.531  267.849  259.745
 259.85   253.181  253.874  255.347     252.351  242.975  265.12   258.404
 256.592  248.026  251.564  257.389     254.352  248.764  259.832  255.358
 260.583  254.463  254.889  248.373  …  247.386  253.957  258.725  252.433
 260.79   255.477  261.272  263.816     260.569  254.48   269.198  266.2  
 255.936  246.098  251.435  249.117     248.44   244.648  262.274  253.384
 254.021  245.83   248.266  251.313     246.582  240.981  255.547  248.779
 253.917  250.891  247.666  253.851     248.124  247.972  255.63   251.605
 264.302  259.857  260.422  263.348  …  262.938  252.856  267.633  263.638
 266.4    263.05   265.823  259.581     262.129  255.302  270.063  266.505
 242.967  241.852  240.739  242.858     242.499  235.089  251.127  243.9

In [35]:
@time begin
    Bref = @spawn rand(1000,1000)^2
    fetch(Bref)
end

  0.086037 seconds (296 allocations: 7.639 MiB)


1000×1000 Array{Float64,2}:
 243.806  243.64   252.726  244.483  …  253.424  248.386  243.628  246.184
 250.03   246.795  256.952  250.109     253.873  258.346  246.432  246.619
 259.294  253.378  262.675  255.474     261.308  259.71   254.755  262.798
 252.465  248.929  258.608  250.546     262.352  255.575  250.516  250.077
 255.096  245.706  260.887  254.912     256.119  254.285  248.993  250.773
 248.316  243.437  254.808  249.25   …  252.433  246.096  240.356  247.763
 247.531  241.044  252.041  251.16      247.67   247.144  239.434  241.214
 258.228  249.972  259.064  254.262     256.265  251.418  246.76   253.417
 258.763  244.008  258.74   250.972     254.936  253.824  253.512  257.931
 255.136  246.773  261.987  260.319     260.321  256.937  247.253  248.99 
 243.768  243.328  257.77   244.062  …  251.221  253.141  241.353  245.661
 254.772  246.124  254.704  249.421     251.301  248.194  242.888  247.089
 248.59   247.096  253.851  249.837     254.093  251.694  246.354  247.4

## Shared Arrays

Shared Arrays are created by mapping the same region in memory to different processes. 

In [19]:
s = SharedArray{Float64}(100,100)

100×100 SharedArray{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0

In [20]:
localindexes(s)

1:0

In [21]:
fetch(@spawnat 2 localindexes(s))

1:5000

In [22]:
fetch(@spawnat 3 localindexes(s))

5001:10000

In [23]:
for i in workers()
    @spawnat i s[localindexes(s)] = myid()
end

In [24]:
s

100×100 SharedArray{Float64,2}:
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0  …  3.0  3.0  3.0  3.0  3.0  3.0  3.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     3.0  3.0  3.0  3.0  3.0  3.0  3.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     3.0  3.0  3.0  3.0  3.0  3.0  3.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     3.0  3.0  3.0  3.0  3.0  3.0  3.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     3.0  3.0  3.0  3.0  3.0  3.0  3.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0  …  3.0  3.0  3.0  3.0  3.0  3.0  3.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     3.0  3.0  3.0  3.0  3.0  3.0  3.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     3.0  3.0  3.0  3.0  3.0  3.0  3.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     3.0  3.0  3.0  3.0  3.0  3.0  3.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     3.0  3.0  3.0  3.0  3.0  3.0  3.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0  …  3.0  3.0  3.0  3.0  3.0  3.0  3.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     3.0  3.0  3.0  3.0  3.0  3.0  3.0
 2.0  2.0  2.0  2.0  2.0  2.0  2

In [25]:
fetch(@spawnat 2 s[100,100])

3.0

In [27]:
fetch(@spawnat 3 s[1,1])

2.0